In [1]:
import pandas as pd

pd.set_option('display.max_columns', 150)

In [2]:
data = pd.read_csv("data/processed_data_2022-06-13.csv")
data.head()

,sofifa_id,player_url,short_name,long_name,overall,potential,value_eur,wage_eur,dob,weight_kg,height_cm,club_team_id,club_position,club_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,release_clause_eur,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,age,Cautious With Crosses,Chip Shot (AI),Comes For Crosses,Dives Into Tackles (AI),Early Crosser,Finesse Shot,Flair,GK Long Throw,Giant Throw-in,Injury Prone,Leadership,Long Passer (AI),Long Shot Taker (AI),Long Throw-in,One Club Player,Outside Foot Shot,Playmaker (AI),Power Free-Kick,Power Header,Rushes Out Of Goal,Saves with Feet,Solid Player,Speed Dribbler (AI),Team Player,Technical Dribbler (AI),#Acrobat,#Aerial Threat,#Clinical Finisher,#Complete Defender,#Complete Forward,#Complete Midfielder,#Crosser,#Distance Shooter,#Dribbler,#Engine,#FK Specialist,#Playmaker,#Playmaker,#Poacher,#Speedster,#Strength,#Tackling,#Tackling,#Tactician,#Tactician,CAM,CB,CDM,CF,CM,GK,LB,LM,LW,LWB,RB,RM,RW,RWB,ST
0,167575,https://sofifa.com/player/167575/anton-shunin/...,A. Shunin,Anton Shunin,76,76,NaN,NaN,1987-01-27,88,191,NaN,NaN,NaN,0,3,1,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18,11,12,14,15,13,30,20,50,39,43,71,48,53,64,30,68,12,36,22,13,47,21,42,16,14,17,77,73,70,75,78,46.0,30,30,30,27,28,28,28,27,31,31,31,30,31,31,31,30,29,31,31,31,29,29,30,30,30,29,76,35,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,205211,https://sofifa.com/player/205211/milan-borjan/...,M. Borjan,Milan Borjan,76,76,NaN,NaN,1987-10-23,84,196,NaN,NaN,NaN,1,2,1,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15,12,24,28,16,16,48,23,47,40,40,73,46,56,68,40,79,12,44,24,14,57,23,60,21,23,21,78,74,74,74,73,44.0,35,35,35,31,33,33,33,31,37,37,37,36,39,39,39,36,33,39,39,39,33,33,37,37,37,33,76,34,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,244700,https://sofifa.com/player/244700/yuriy-dyupin/...,Y. Dyupin,Yuriy Dyupin,74,74,NaN,NaN,1988-03-17,84,187,NaN,NaN,NaN,1,3,1,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12,6,10,7,14,11,22,12,20,21,27,63,43,51,59,27,72,5,20,6,5,31,16,56,9,13,13,76,71,68,74,78,20.0,24,24,24,17,19,19,19,17,22,22,22,21,23,23,23,21,21,24,24,24,21,20,24,24,24,20,74,34,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,199904,https://sofifa.com/player/199904/jesse-joronen...,J. Joronen,Jesse Pekka Joronen,72,72,NaN,NaN,1993-03-21,89,197,NaN,NaN,NaN,1,3,1,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15,13,18,16,14,12,25,16,48,45,50,70,36,51,37,33,64,13,39,19,14,45,14,61,19,14,15,70,70,68,71,72,47.0,30,30,30,26,28,28,28,26,29,29,29,29,29,29,29,29,28,29,29,29,28,28,29,29,29,28,72,29,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,240225,https://sofifa.com/player/240225/matvey-safono...,M. Safonov,Matvey Safonov,72,82,NaN,NaN,1999-02-25,82,192,NaN,NaN,NaN,1,2,1,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13,7,14,10,11,12,26,18,19,22,33,38,26,55,69,26,64,8,27,13,7,55,11,55,25,13,14,74,74,73,75,76,20.0,23,23,23,19,21,21,21,19,25,25,25,23,27,27,27,23,21,26,26,26,21,21,27,27,27,21,73,23,0,0,0,0,0,0,0

In [3]:
def count_na(df: pd.DataFrame) -> None:
    nan_sum = df.isna().sum()
    display(nan_sum[nan_sum > 0])

count_na(data)

value_eur                74
wage_eur                 61
club_team_id             61
club_position            61
club_jersey_number       61
release_clause_eur     1176
pace                   2132
shooting               2132
passing                2132
dribbling              2132
defending              2132
physic                 2132
goalkeeping_speed     17107
dtype: int64

Analisando essa tabela com os valores nulos podemos perceber que alguns jogadores do dataset não possuem um time atual, isso explica
`wage_eur`, `club_team_id`, `club_position`, `club_jersey_number` terem a mesma quantidade de valores nulos (61),
porém `value_eur` deveria seguir esse mesmo padrão, assim podemos dizer que 13 (74 - 61) jogadores que estão ativos não possuem um valor,
portanto precisamos prever esses dados.

Também podemos perceber que temos 2132 jogadores não possuem `pace`, `shooting`, `passing`, `dribbling`, `defending`, `physic`. Vamos analisar esses jogadores

In [4]:
def percent_of_gk(df: pd.DataFrame) -> None:
    gk = df[df['GK'] == 1]
    display(gk.shape[0] / df.shape[0])


missing_main_attr_data = data[data.pace.isna()]

percent_of_gk(missing_main_attr_data)
missing_main_attr_data

1.0

,sofifa_id,player_url,short_name,long_name,overall,potential,value_eur,wage_eur,dob,weight_kg,height_cm,club_team_id,club_position,club_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,release_clause_eur,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,age,Cautious With Crosses,Chip Shot (AI),Comes For Crosses,Dives Into Tackles (AI),Early Crosser,Finesse Shot,Flair,GK Long Throw,Giant Throw-in,Injury Prone,Leadership,Long Passer (AI),Long Shot Taker (AI),Long Throw-in,One Club Player,Outside Foot Shot,Playmaker (AI),Power Free-Kick,Power Header,Rushes Out Of Goal,Saves with Feet,Solid Player,Speed Dribbler (AI),Team Player,Technical Dribbler (AI),#Acrobat,#Aerial Threat,#Clinical Finisher,#Complete Defender,#Complete Forward,#Complete Midfielder,#Crosser,#Distance Shooter,#Dribbler,#Engine,#FK Specialist,#Playmaker,#Playmaker,#Poacher,#Speedster,#Strength,#Tackling,#Tackling,#Tactician,#Tactician,CAM,CB,CDM,CF,CM,GK,LB,LM,LW,LWB,RB,RM,RW,RWB,ST
0,167575,https://sofifa.com/player/167575/anton-shunin/...,A. Shunin,Anton Shunin,76,76,NaN,NaN,1987-01-27,88,191,NaN,NaN,NaN,0,3,1,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18,11,12,14,15,13,30,20,50,39,43,71,48,53,64,30,68,12,36,22,13,47,21,42,16,14,17,77,73,70,75,78,46.0,30,30,30,27,28,28,28,27,31,31,31,30,31,31,31,30,29,31,31,31,29,29,30,30,30,29,76,35,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,205211,https://sofifa.com/player/205211/milan-borjan/...,M. Borjan,Milan Borjan,76,76,NaN,NaN,1987-10-23,84,196,NaN,NaN,NaN,1,2,1,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15,12,24,28,16,16,48,23,47,40,40,73,46,56,68,40,79,12,44,24,14,57,23,60,21,23,21,78,74,74,74,73,44.0,35,35,35,31,33,33,33,31,37,37,37,36,39,39,39,36,33,39,39,39,33,33,37,37,37,33,76,34,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,244700,https://sofifa.com/player/244700/yuriy-dyupin/...,Y. Dyupin,Yuriy Dyupin,74,74,NaN,NaN,1988-03-17,84,187,NaN,NaN,NaN,1,3,1,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12,6,10,7,14,11,22,12,20,21,27,63,43,51,59,27,72,5,20,6,5,31,16,56,9,13,13,76,71,68,74,78,20.0,24,24,24,17,19,19,19,17,22,22,22,21,23,23,23,21,21,24,24,24,21,20,24,24,24,20,74,34,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,199904,https://sofifa.com/player/199904/jesse-joronen...,J. Joronen,Jesse Pekka Joronen,72,72,NaN,NaN,1993-03-21,89,197,NaN,NaN,NaN,1,3,1,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15,13,18,16,14,12,25,16,48,45,50,70,36,51,37,33,64,13,39,19,14,45,14,61,19,14,15,70,70,68,71,72,47.0,30,30,30,26,28,28,28,26,29,29,29,29,29,29,29,29,28,29,29,29,28,28,29,29,29,28,72,29,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,240225,https://sofifa.com/player/240225/matvey-safono...,M. Safonov,Matvey Safonov,72,82,NaN,NaN,1999-02-25,82,192,NaN,NaN,NaN,1,2,1,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13,7,14,10,11,12,26,18,19,22,33,38,26,55,69,26,64,8,27,13,7,55,11,55,25,13,14,74,74,73,75,76,20.0,23,23,23,19,21,21,21,19,25,25,25,23,27,27,27,23,21,26,26,26,21,21,27,27,27,21,73,23,0,0,0,0,0,0,0

Como vimos que os goleiros não tem os atributos listados acima, mas possuem um overall em cada posição, seria interessante prever esses atributos.

In [5]:
missing_gk_speed_data = data[data.goalkeeping_speed.isna()]
sum_of_missing_data = missing_gk_speed_data.shape[0] + missing_main_attr_data.shape[0]
print(f"Jogadores sem gk speed + jogadores sem atributos primários: {sum_of_missing_data}")
print(f"Numero de jogadores: {data.shape[0]}")

Jogadores sem gk speed + jogadores sem atributos primários: 19239
Numero de jogadores: 19239


Assim podemos ver que todos os jogadores que não são goleiros não possuem `goalkeeping_speed`

## Prevendo valor dos jogadores
Nessa etapa vamos criar um modelo para prever o valor dos 13 jogadores que não possuem valor

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import numpy as np

def predict_and_populate(df_to_populate, df_to_train, model, column_to_predict, round_=None) -> pd.DataFrame:
    # Separating the target variable
    target = df_to_train[column_to_predict]
    features = df_to_train.drop(column_to_predict, axis=1)

    # Poluting the missing features
    mean_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
    mean_imputer.fit(features)
    imputed_features = mean_imputer.transform(features)

    # Train test split
    features_train, features_test, target_train, target_test = train_test_split(imputed_features, target, test_size=0.2)

    # Training the model
    trained_model = model.fit(features_train, target_train)

    # Calculatin model score
    model_score = trained_model.score(features_test, target_test)
    print(f"Score do modelo: {model_score:.4f}")

    # Populating the missing values
    feature_to_populate = mean_imputer.transform(df_to_populate.drop(column_to_predict, axis=1))
    predicted_values = trained_model.predict(feature_to_populate)

    if round_ is not None:
        predicted_values = predicted_values.round(round_)

    df_to_populate[column_to_predict] = predicted_values

    return df_to_populate

In [7]:
players_with_club_and_no_value = data.query("not club_team_id.isna() and value_eur.isna()").copy()
data_without_players_with_club_and_no_value = data.drop(players_with_club_and_no_value.index)

data_to_train = data_without_players_with_club_and_no_value.query("not value_eur.isna()").copy()
columns_to_drop = ["sofifa_id", "player_url", "short_name", "long_name", "dob", "club_position"]


data_to_train = data_to_train.drop(columns=columns_to_drop)
data_to_fill = players_with_club_and_no_value.drop(columns=columns_to_drop)

In [8]:
# Normalizando os dados
# from sklearn.preprocessing import StandardScaler
#
# standerd_scaler =  StandardScaler()
# standardized_features = standerd_scaler.fit_transform(imputed_features)
# standardized_features

## Comparando modelos diferentes

In [9]:
from sklearn.dummy import DummyRegressor

dummy = DummyRegressor()
predict_and_populate(data_to_fill, data_to_train, dummy, "value_eur");

Score do modelo: -0.0001


In [10]:
from sklearn.linear_model import LinearRegression

linear_regression = LinearRegression()
predict_and_populate(data_to_fill, data_to_train, linear_regression, "value_eur");

Score do modelo: 0.9752


In [11]:
from sklearn.ensemble import RandomForestRegressor

random_forest = RandomForestRegressor(random_state=0, n_jobs=-1, oob_score=True)
players_with_club_and_no_value = predict_and_populate(data_to_fill, data_to_train, random_forest, "value_eur");

Score do modelo: 0.9923


### Preenchendo os dados a partir do modelo criado

In [12]:
data.loc[players_with_club_and_no_value.index, "value_eur"] = players_with_club_and_no_value.value_eur
data.loc[players_with_club_and_no_value.index]

,sofifa_id,player_url,short_name,long_name,overall,potential,value_eur,wage_eur,dob,weight_kg,height_cm,club_team_id,club_position,club_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,release_clause_eur,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,age,Cautious With Crosses,Chip Shot (AI),Comes For Crosses,Dives Into Tackles (AI),Early Crosser,Finesse Shot,Flair,GK Long Throw,Giant Throw-in,Injury Prone,Leadership,Long Passer (AI),Long Shot Taker (AI),Long Throw-in,One Club Player,Outside Foot Shot,Playmaker (AI),Power Free-Kick,Power Header,Rushes Out Of Goal,Saves with Feet,Solid Player,Speed Dribbler (AI),Team Player,Technical Dribbler (AI),#Acrobat,#Aerial Threat,#Clinical Finisher,#Complete Defender,#Complete Forward,#Complete Midfielder,#Crosser,#Distance Shooter,#Dribbler,#Engine,#FK Specialist,#Playmaker,#Playmaker,#Poacher,#Speedster,#Strength,#Tackling,#Tackling,#Tactician,#Tactician,CAM,CB,CDM,CF,CM,GK,LB,LM,LW,LWB,RB,RM,RW,RWB,ST
11929,115909,https://sofifa.com/player/115909/ruben-castro-...,Rubén Castro,Rubén Castro Martín,69,69,905000.0,3000.0,1981-06-27,68,169,100851.0,ST,7.0,1,3,3,2,0,0,NaN,70.0,70.0,61.0,72.0,29.0,61.0,60,71,62,70,68,67,45,69,72,69,86,75,81,69,85,62,61,68,54,37,77,74,75,68,36,12,14,14,8,15,8,12,NaN,69,69,69,69,69,69,69,69,69,69,69,69,63,63,63,69,52,49,49,49,52,49,45,45,45,49,20,40,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
11987,124344,https://sofifa.com/player/124344/ibrahim-oztur...,İ. Öztürk,İbrahim Öztürk,67,67,801750.0,3000.0,1981-06-21,80,186,101006.0,CB,38.0,1,2,2,1,0,0,NaN,32.0,43.0,48.0,48.0,68.0,74.0,44,30,72,46,44,48,42,54,30,34,32,68,38,66,59,65,78,46,78,66,34,44,51,51,70,68,62,15,13,10,10,6,NaN,52,52,52,44,47,47,47,44,48,48,48,48,54,54,54,48,59,64,64,64,59,61,67,67,67,61,19,40,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
12264,153066,https://sofifa.com/player/153066/lucas-licht/2...,L. Licht,Lucas Matías Licht,69,69,921000.0,5000.0,1981-04-06,72,174,101084.0,SUB,25.0,0,4,3,1,0,0,NaN,64.0,61.0,65.0,72.0,68.0,66.0,80,52,53,76,73,68,71,69,59,68,69,66,73,71,72,64,66,68,69,71,56,64,84,72,69,71,69,8,14,6,15,8,NaN,63,63,63,65,63,63,63,65,66,66,66,68,67,67,67,68,69,69,69,69,69,69,69,69,69,69,19,41,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0
13149,152912,https://sofifa.com/player/152912/jose-sand/220002,J. Sand,José Gustavo Sand,75,75,3751000.0,13000.0,1980-07-17,79,182,110395.0,LS,9.0,1,4,3,1,0,0,NaN,51.0,79.0,62.0,68.0,37.0,70.0,53,81,77,67,63,63,46,75,49,53,38,81,67,77,71,41,83,73,73,33,83,69,85,85,45,21,25,11,16,15,9,14,NaN,75,75,75,68,73,73,73,68,72,72,72,68,65,65,65,68,51,54,54,54,51,49,53,53,53,49,22,41,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
13177,110381,https://sofifa.com/player/110381/maximiliano-r...,M. Rodríguez,Maximiliano Rubén Rodríguez,73,73,2380750.0,10000.0,1981-01-02,74,173,110396.0,SUB,11.0,1,4,4,3,0,0,NaN,64.0,74.0,76.0,75.0,44.0,64.0,72,73,61,75,76,71,75,76,64,64,72,72,70,77,67,46,70,75,

In [13]:
count_na(data)

value_eur                61
wage_eur                 61
club_team_id             61
club_position            61
club_jersey_number       61
release_clause_eur     1176
pace                   2132
shooting               2132
passing                2132
dribbling              2132
defending              2132
physic                 2132
goalkeeping_speed     17107
dtype: int64

In [14]:
# Removendo jogadores não ativos
data = data.query("not club_team_id.isna()").copy()

count_na(data)

release_clause_eur     1115
pace                   2124
shooting               2124
passing                2124
dribbling              2124
defending              2124
physic                 2124
goalkeeping_speed     17054
dtype: int64

In [15]:
# TODO acidionar .joblib ou .pickle do modelo

# Prevendo pace, shooting, passing, dribbling, defending, physic dos goleiros

In [16]:
gk_data = data.query("GK == 1").copy()
data_without_gk = data.query("GK == 0").copy()

columns_to_predict = ["pace", "shooting", "passing", "dribbling", "defending", "physic"]

for column_to_predict in columns_to_predict:

    columns_to_drop_with_other_atributes = [col for col in columns_to_predict if col != column_to_predict]
    columns_to_drop_with_other_atributes.extend(columns_to_drop)

    data_to_train = data_without_gk.drop(columns=columns_to_drop_with_other_atributes)
    data_to_predict = gk_data.drop(columns=columns_to_drop_with_other_atributes)

    linear_regression = LinearRegression()
    print(f"Prevendo {column_to_predict}")
    filled_data = predict_and_populate(data_to_predict, data_to_train, linear_regression, column_to_predict, 0)
    print()

    data.loc[filled_data.index, column_to_predict] = filled_data[column_to_predict]

# data.loc[gk_data.index]

Prevendo pace
Score do modelo: 0.9993

Prevendo shooting
Score do modelo: 0.9989

Prevendo passing
Score do modelo: 0.9961

Prevendo dribbling
Score do modelo: 0.9991

Prevendo defending
Score do modelo: 0.9997

Prevendo physic
Score do modelo: 0.9991



In [17]:
count_na(data)

release_clause_eur     1115
goalkeeping_speed     17054
dtype: int64

## Prevendo goalkeeping_speed dos jogadores

In [18]:
players_data = data.query("GK == 0").copy()
players_data

,sofifa_id,player_url,short_name,long_name,overall,potential,value_eur,wage_eur,dob,weight_kg,height_cm,club_team_id,club_position,club_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,release_clause_eur,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,age,Cautious With Crosses,Chip Shot (AI),Comes For Crosses,Dives Into Tackles (AI),Early Crosser,Finesse Shot,Flair,GK Long Throw,Giant Throw-in,Injury Prone,Leadership,Long Passer (AI),Long Shot Taker (AI),Long Throw-in,One Club Player,Outside Foot Shot,Playmaker (AI),Power Free-Kick,Power Header,Rushes Out Of Goal,Saves with Feet,Solid Player,Speed Dribbler (AI),Team Player,Technical Dribbler (AI),#Acrobat,#Aerial Threat,#Clinical Finisher,#Complete Defender,#Complete Forward,#Complete Midfielder,#Crosser,#Distance Shooter,#Dribbler,#Engine,#FK Specialist,#Playmaker,#Playmaker,#Poacher,#Speedster,#Strength,#Tackling,#Tackling,#Tactician,#Tactician,CAM,CB,CDM,CF,CM,GK,LB,LM,LW,LWB,RB,RM,RW,RWB,ST
2185,188567,https://sofifa.com/player/188567/pierre-emeric...,P. Aubameyang,Pierre-Emerick Emiliano François Aubameyang,85,85,43500000.0,145000.0,1989-06-18,80,187,1.0,ST,14.0,1,4,4,3,0,0,82700000.0,89.0,85.0,75.0,80.0,35.0,69.0,76,89,73,79,81,76,64,81,87,91,77,86,67,82,75,79,74,78,43,48,90,77,83,87,20,29,36,6,9,15,9,9,NaN,85,85,85,82,83,83,83,82,84,84,84,84,77,77,77,84,67,61,61,61,67,63,54,54,54,63,20,32,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1
2186,209989,https://sofifa.com/player/209989/thomas-partey...,T. Partey,Thomas Partey,83,83,36500000.0,115000.0,1993-06-13,77,185,1.0,RDM,5.0,1,3,3,2,0,0,69400000.0,69.0,71.0,82.0,81.0,80.0,84.0,74,66,68,84,69,70,85,83,63,74,75,82,50,86,83,83,84,78,84,81,70,85,49,86,80,84,80,11,13,9,9,7,NaN,77,77,77,78,78,78,78,78,82,82,82,81,83,83,83,81,82,83,83,83,82,81,83,83,83,81,20,29,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0
2187,193301,https://sofifa.com/player/193301/alexandre-lac...,A. Lacazette,Alexandre Lacazette,82,82,30000000.0,110000.0,1991-05-28,73,175,1.0,SUB,9.0,1,4,4,3,1,0,57000000.0,75.0,82.0,74.0,82.0,44.0,72.0,68,83,75,83,74,74,70,82,77,73,81,82,82,87,76,66,75,78,73,42,82,75,79,80,39,45,32,11,6,9,5,6,NaN,82,82,82,79,81,81,81,79,82,82,82,81,77,77,77,81,64,65,65,65,64,62,60,60,60,62,18,31,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2188,222665,https://sofifa.com/player/222665/martin-odegaa...,M. Ødegaard,Martin Ødegaard,82,88,49000000.0,89000.0,1998-12-17,68,178,1.0,CAM,8.0,0,2,5,3,0,0,102900000.0,77.0,74.0,83.0,84.0,58.0,62.0,84,73,52,85,81,83,81,85,80,75,85,74,83,76,46,78,56,77,63,65,76,86,63,76,65,49,51,14,15,15,8,10,NaN,77,77,77,81,79,79,79,81,84,84,84,84,82,82,82,84,75,73,73,73,75,71,63,63,63,71,22,23,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2189,226110,https://sofifa.com/player/226110/nicolas-pepe/...,N. Pépé,Nicolas Pépé,81,82,30500000.0,95000.0,1995-05-29,73,183,1.0,RM,19.0,0,2,4,2,0,0,60200000.0,88.0,79.0,75.0,83.0,36.0,65.0,77,79,57,85,81,79,66,81,88,8

In [19]:
data_to_predict = players_data.drop(columns=columns_to_drop)
data_to_train = gk_data.drop(columns=columns_to_drop)

column_to_predict = "goalkeeping_speed"

linear_regression = LinearRegression()
filled_data = predict_and_populate(data_to_predict, data_to_train, linear_regression, column_to_predict, 0)

data.loc[filled_data.index, column_to_predict] = filled_data[column_to_predict]

Score do modelo: 0.9993


In [20]:
count_na(data)

release_clause_eur    1115
dtype: int64

In [21]:
import os
def get_project_root_path():
    project_directory = "fifa22-players-analysis"
    separated_path = os.getcwd().split(os.sep)
    project_directory_index = separated_path.index(project_directory) + 1
    return os.sep.join(separated_path[0:project_directory_index])

In [22]:
from datetime import date
data.drop(columns=["release_clause_eur"], inplace=True)

filled_data_save_path = os.sep.join([get_project_root_path(), "preprocessing", "data", f"filled_data_{date.today()}.csv"])
data.to_csv(filled_data_save_path, index=False)
filled_data_save_path

'/home/gustavo/fifa22-players-analysis/preprocessing/data/filled_data_2022-06-14.csv'

In [23]:
count_na(data)

Series([], dtype: int64)